In [1]:
using CSV, Tables, LinearAlgebra, Random, Gurobi, JuMP, Statistics, DataFrames, JLD2

# Import parameters

In [6]:
# Van information
# Number of vans
K=2
# Capacity of vans
S=10

# Station information
stations_info = CSV.read("../data/stations/station_information.csv", DataFrame);

# Number of stations
n_stations = size(stations_info, 1);
n_stations_test = 100

# Capacity
C = stations_info[!, :capacity];
C_test=C[1:n_stations_test,:];

# Initial state 
y0 = floor.(C/2);
y0_test=y0[1:n_stations_test];

# Distances
D = load("../data/parameters/distances.jld2", "D");
d_test=D[1:n_stations_test,1:n_stations_test];

# Feasibility
X = load("../data/parameters/feasibility.jld2", "X");
X_test=X[1:n_stations_test,1:n_stations_test];

# Demand
demand = load("../data/parameters/202210-demand.jld2", "demand");
d_test=demand[1:n_stations_test,1:n_stations_test,:];

# Demand over one day
demand_1day = demand[:,:,1:24];
demand_1day_test = d_test[:, :, 1:24];

In [16]:
# create empty list of size n_stations_test
stations_score = zeros(n_stations)
# for each station i, sum the number of bikes that gets in an out ot the station over the month
for i in 1:n_stations
    stations_score[i] += sum(demand_1day[i,:,:]) + sum(demand_1day[:,i,:])
end
stations_score;

# sort stations by score
stations_score_sorted = sortperm(stations_score, rev=true);

# reorder index of columns and rows of demand according to the sorted stations
demand_1day_sorted = demand_1day[stations_score_sorted, stations_score_sorted, :];
# reorder index of capacity according to the sorted stations
C_sorted = C[stations_score_sorted,:];
# reorder index of initial state according to the sorted stations
y0_sorted = y0[stations_score_sorted,:];
# reorder index of distances according to the sorted stations
D_sorted = D[stations_score_sorted, stations_score_sorted];
# reorder index of feasibility according to the sorted stations
X_sorted = X[stations_score_sorted, stations_score_sorted];

# Let's reduce the number of stations to reduce complexity
nb=200
demand_1day_filtered = demand_1day_sorted[1:nb, 1:nb, :];
C_filtered = C_sorted[1:nb,:];
y0_filtered = y0_sorted[1:nb,:];
D_filtered = D_sorted[1:nb, 1:nb];
X_filtered = X_sorted[1:nb, 1:nb];

# Model

In [18]:
function first_model(C,D,X,y0,K,S,d)
    model = Model(Gurobi.Optimizer)
    set_optimizer_attribute(model, "OutputFlag", 0)
    set_optimizer_attribute(model, "TimeLimit", 100)

    n_stations, _, n_hours = size(d)
    M=30
    lambda=0.01 # to be tuned
    n_rebal_stations = 50 # to be tuned
    # Decision variables
    @variable(model, x[1:n_rebal_stations, 1:n_rebal_stations, 1:K, 1:n_hours], Bin)
    @variable(model, 0 <= z[1:n_rebal_stations, 1:n_rebal_stations, 1:K, 1:n_hours], Int)
    @variable(model, 0 <= y[1:n_stations, 1:n_hours]) # Int by definition of the constraints
    @variable(model, 0 <= w[1:n_stations, 1:n_stations, 1:n_hours], Int)
    @variable(model, 0 <= u[1:n_stations, 1:n_stations, 1:n_hours])

    # Add constraints:
    # Stations capacity
    @constraint(model, [i in 1:n_stations, t in 1:n_hours], y[i,t] <= C[i])
    # No rebalancing at the same station
    @constraint(model, [i in 1:n_rebal_stations, k in 1:K, t in 1:n_hours], x[i,i,k,t] == 0)
    # Flow balance 
    @constraint(model, [i in 1:n_rebal_stations, t in 2:n_hours], y[i,t] - y[i,t-1] == sum(w[j,i,t] for j in 1:n_stations)-sum(w[i,j,t] for j in 1:n_stations)-sum(z[i,j,k,t] for j in 1:n_rebal_stations, k in 1:K)+sum(z[j,i,k,t] for j in 1:n_rebal_stations, k in 1:K))
    @constraint(model, [i in n_rebal_stations+1:n_stations, t in 2:n_hours], y[i,t] - y[i,t-1] == sum(w[j,i,t] for j in 1:n_stations)-sum(w[i,j,t] for j in 1:n_stations))
    # Flow balance for the first hour 
    @constraint(model, [i in 1:n_rebal_stations], y[i,1] - y0[i] == sum(w[j,i,1] for j in 1:n_stations)-sum(w[i,j,1] for j in 1:n_stations)-sum(z[i,j,k,1] for j in 1:n_rebal_stations, k in 1:K)+sum(z[j,i,k,1] for j in 1:n_rebal_stations, k in 1:K))
    @constraint(model, [i in n_rebal_stations+1:n_stations], y[i,1] - y0[i] == sum(w[j,i,1] for j in 1:n_stations)-sum(w[i,j,1] for j in 1:n_stations))
    # Vans capacity
    @constraint(model, [i in 1:n_rebal_stations, j in 1:n_rebal_stations, k in 1:K, t in 1:n_hours], z[i,j,k,t] <= S)
    # Users travel availability
    @constraint(model, [i in 1:n_stations, t in 1:n_hours], -y[i,t] <= sum(w[j,i,t] for j in 1:n_stations) - sum(w[i,j,t] for j in 1:n_stations))
    @constraint(model, [i in 1:n_stations, t in 1:n_hours], sum(w[j,i,t] for j in 1:n_stations) - sum(w[i,j,t] for j in 1:n_stations)<= C[i] - y[i,t])
    # Max 1 rebalancing per hour per van
    @constraint(model, [k in 1:K, t in 1:n_hours], sum(x[i,j,k,t] for i in 1:n_rebal_stations, j in 1:n_rebal_stations) <= 1)
    # Rebalancing feasibility
    @constraint(model, [i in 1:n_rebal_stations, j in 1:n_rebal_stations, k in 1:K, t in 1:n_hours], z[i,j,k,t] <= M*x[i,j,k,t])
    @constraint(model, [i in 1:n_rebal_stations, j in 1:n_rebal_stations, k in 1:K, t in 1:n_hours], x[i,j,k,t] <= X[i,j])
    # Vans must travel sequentially
    @constraint(model, [i in 1:n_rebal_stations,l in 1:n_rebal_stations, k in 1:K, t in 2:n_hours], sum(x[i,j,k,t] for j in 1:n_rebal_stations) <= x[l,i,k,t-1])
    # Users travel is inferior to the demand and definition of u
    @constraint(model, [i in 1:n_stations, j in 1:n_stations, t in 1:n_hours], u[i,j,t] >= d[i,j,t] - w[i,j,t])
    @constraint(model, [i in 1:n_stations, j in 1:n_stations, t in 1:n_hours], d[i,j,t] >= w[i,j,t])

    # Set objective
    @objective(model, Min, sum(u[i,j,t] for i in 1:n_stations, j in 1:n_stations, t in 1:n_hours)+lambda*sum(D[i,j]*x[i,j,k,t] for i in 1:n_rebal_stations, j in 1:n_rebal_stations, k in 1:K, t in 1:n_hours))
    
    # Solve the model
    optimize!(model)
    
    # Print the solution
    println("Objective value: ", objective_value(model))
    return value.(x), value.(w), value.(u), value.(y), value.(z), objective_value(model)
end

first_model (generic function with 1 method)

In [19]:
# x,w,u,y,z,obj=first_model(C,D,X,y0,K,S,demand_1day);
x,w,u,y,z,obj=first_model(C_filtered,D_filtered,X_filtered,y0_filtered,K,S,demand_1day_filtered);

Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-15
Set parameter TimeLimit to value 100
Objective value: 987.0067295485362


In [151]:
sum(x)

11.0

In [148]:
sum(sum(demand_1day-w, dims=2),dims=3) |> vec

449-element Vector{Float64}:
  2.0
  5.0
  2.0
  3.0
  1.0
  5.0
 56.0
 18.0
  0.0
  0.0
  1.0
  1.0
  3.0
  ⋮
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  5.0
  0.0
  0.0
  0.0
  0.0

In [147]:
# total outgoing demand per day for each station
mostsortperm(sum(sum(demand_1day-w, dims=2),dims=3) |> vec)


449-element Vector{Int64}:
   9
  10
  30
  31
  35
  37
  41
  42
  43
  44
  52
  54
  57
   ⋮
   8
  46
  87
  48
 420
 296
 153
  28
  59
  66
  60
   7